In [2]:
import pandas as pd
import os

In [3]:
def isEnglishOrKorean(input):
    for c in input:
        if ord('가') <= ord(c) <= ord('힣'):
            return False

    return True

In [9]:
# def sqlite_to_pandas():
DATA_DIR = "."
DUMP_FILE = os.path.join(DATA_DIR, "all_data.pkl")
DUMP_FILE_EN = os.path.join(DATA_DIR, "title_en.pkl")
DUMP_FILE_KO = os.path.join(DATA_DIR, "title_ko.pkl")

data = pd.read_pickle(DUMP_FILE)
result_ko = []
result_en = []

for idx in range(0, len(data)):
    if data['title_kor'][idx] is None:
        continue
        
    if isEnglishOrKorean(data['title_kor'][idx]):
        result_en.append(data.iloc[idx, :])
    else:
        result_ko.append(data.iloc[idx, :])

endf = pd.DataFrame.from_records(data=result_en, columns=data.columns)
kodf = pd.DataFrame.from_records(data=result_ko, columns=data.columns)
pd.to_pickle(endf, DUMP_FILE_EN)
pd.to_pickle(kodf, DUMP_FILE_KO)
print(len(result_en))
print(len(result_ko))

33722
41811


In [10]:
### 데이터 전처리
### 1. label번호 바꾸기 (db랑 비교)
### 2. train_data / test_data 나누기

data_en = pd.read_pickle(DUMP_FILE_EN)
data_ko = pd.read_pickle(DUMP_FILE_KO)

subjects = ["기타자연과학", "대기과학", "물리학", "생물학", "생활과학", "수학", "자연과학", "자연과학일반", "지구과학", "지질학", "천문학", "통계학", "해양학", "화학", "건축공학", "고분자공학", "공학", "공학일반", "교통공학", "금속공학", "기계공", 
"기타공학", "농공학", "산림공학", "산업공학", "생물공학", "섬유공학", "안전공학", "원자력공학", "의공학", "자동차공학", "자원공학", "재료공학", "전기공학", "전자/정보통신공학", "제어계측공학", "조선공학", "컴퓨터학", "토목공학", "항공우주공학", "해양공학", "화학공학", "환경공학"
]

results = []
for idx in range(0, len(data_en)):
    if data_en['subject'][idx] is None:
        results.append(0)
        continue
    
    i = 0
    flag = True
    for subject in subjects:
        i += 1
        if data_en['subject'][idx] == subject:
#             print(data_en['subject'][idx])
            flag = False
            results.append(i)
            break
    
    if flag is True:
        results.append(0)

data_en['label'] = results
print(data_en.head(20))


    id                                          title_kor title_eng  \
0    1  Characterization of Electrical Heating of Grap...             
1    2  The textile products labelling analysis and re...             
2    3  Consumer resistance to innovation: smart clothing             
3    4  Experimental design and evaluation of a moistu...             
4    5  Application of persimmon (Diospyros kaki L.) p...             
5    6  Constructing cultural identity through weaving...             
6    7  On the fashionalization of digital devices: a ...             
7    8  Top online luxury apparel and accessories reta...             
8    9  Changing structures of B2B networks in the Jap...             
9   10  Predicting clothing behaviors of Generation Y ...             
10  11  Product experiences of clothing attachment in ...             
11  12  Physiological and subjective burden when weari...             
12  13  ‘Neo-Crosssexual’ fashion in contemporary men’...             
13  14

In [11]:
df = data_en.loc[:, ["title_kor", "label"]]

# 특수 문자, 길이 짧은 단어, 소문자 변환
df["title_clean"] = df["title_kor"].str.replace("[^a-zA-Z]", " ")
df["title_clean"] = df["title_clean"].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
df["title_clean"] = df["title_clean"].apply(lambda x: x.lower())

# Null 값 및 empty값 확인
df["title_clean"].isnull().values.any()
df.replace("", float("NaN"), inplace=True)
df["title_clean"].isnull().values.any()
# print(df.head(1))

True

In [42]:
from nltk.corpus import stopwords
import numpy as np
# from tensorflow.keras.preprocessing.text import Tokenizer

# Null 값 제거
print('총 샘플 수: ', len(df))
df.dropna(inplace=True)
df = df[df.label != 0]
print('총 샘플 수: ', len(df))

# 불용어 제거
stop_words = stopwords.words('english')
tokenized_doc = df['title_clean'].apply(lambda x: x.split())
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
tokenized_doc = tokenized_doc.to_list()

# 단어가 1개 이하인 샘플의 인덱스를 찾아서 저장하고, 해당 샘플들은 제거.
# drop_train = [index for index, sentence in enumerate(tokenized_doc) if len(sentence) <= 1]
# tokenized_doc = np.delete(tokenized_doc, drop_train, axis=0)
print('총 샘플 수 :',len(tokenized_doc))

총 샘플 수:  33721
총 샘플 수:  30429
총 샘플 수 : 30429


In [48]:
### Splitting Data
### https://wikidocs.net/33274
import numpy as np
from sklearn.model_selection import train_test_split

df['tokenized_doc'] = tokenized_doc

X_train, X_test, y_train, y_test = train_test_split(df['tokenized_doc'], df['label'], test_size= 0.2, random_state=1234)
print(X_train)
print(X_test)
print(y_train)
print(y_test)

# null 값 제거
# train_data = train_data.dropna(how='any')
# test_data = test_data.dropna(how='any')



4802     [activation, unfolded, protein, response, expr...
17453    [aqueous, synthesis, photocatalytic, performan...
12752    [smartphone, optical, physical, electrochemica...
13886    [time, varying, barrier, lyapunov, function, b...
15974    [effect, varying, beam, reinforcement, anchora...
                               ...                        
30904    [current, limit, strategy, voltage, controller...
26176    [path, tracking, autonomous, race, based, diag...
1511     [enhancement, rssi, based, localization, using...
28065    [attitude, determination, algorithm, based, re...
30427    [sims, study, artificially, photoaged, silk, f...
Name: tokenized_doc, Length: 24343, dtype: object
28090    [crotonaldehyde, removal, polluted, using, bio...
32038    [influence, hydrophobicity, acetylated, nanoce...
25363    [comparative, study, three, different, viabili...
18775    [design, virtual, reality, whole, body, vibrat...
22441    [improving, subject, independent, classificati...
      

In [49]:
# 텍스트 단어 부호화(정수 인코딩)
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=30000)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

print(X_train)
print(X_test)

[[490, 8923, 193, 73, 115, 175, 1467, 115, 1100, 8924, 3591, 3916], [263, 22, 274, 12, 8925], [2176, 128, 386, 211, 12444], [27, 431, 869, 2367, 121, 1, 106, 91, 345, 14, 75, 1601, 526, 16, 1643, 2604], [6, 431, 102, 688, 3592, 8926, 3338, 102, 300, 456], [1333, 52, 127, 246, 301, 1334, 1000, 331, 82], [257, 312, 3593, 2605, 36, 2, 21, 531, 260, 1128, 223, 689], [6, 2177, 270, 349, 35, 5, 47, 157, 2177], [141, 74, 85, 18, 197, 88, 974, 12445, 7], [4271, 2178, 2763, 202, 1750, 627, 541, 7148], [12446, 1297, 52, 490, 390, 228, 108, 438, 602, 1335, 123, 24], [2933, 1701, 5, 12447, 12448, 911], [4, 14, 37, 1069, 1644, 319, 1930, 325, 4736, 5328], [1070, 7149, 12449, 652, 3594, 13, 1070, 12450, 652, 355, 113, 1549, 439, 113, 1549, 2764, 12451, 12452], [31, 135, 577, 127, 11, 2, 1193, 1, 75, 99], [816, 5, 218, 1261, 2275, 4737, 241, 2368, 229, 2765, 218, 220], [1262, 198, 2, 590, 1263, 32, 1226, 236, 27, 483, 1468], [2606, 6089, 3339, 6090, 115, 432, 67, 12453], [3117, 12454, 5329, 10, 7150,

In [45]:
#### 딥러닝
from keras.layers import Embedding, Dense, LSTM
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences


# X_train, X_test, y_train, y_test 
# 데이터 준비
# train_label = train_data['label']
# test_label = test_data['label']

# 데이터 정규화
X_train = pad_sequences(X_train,maxlen=30)
X_test = pad_sequences(X_test,maxlen=30)

# 모델 생성
model = Sequential()

model.add(Embedding(30000,100))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

# 모델 설정
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# 모델 학습
model.fit(X_train,
          y_train,
          epochs=1000,
          batch_size=10,
          validation_split=0.2)

# 모델 평가
result = model.evaluate(X_test, y_test)
print('loss =', result[0])
print('accuracy=', result[1])

C:\ProgramData\Anaconda3\envs\venv\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 19474 samples, validate on 4869 samples
Epoch 1/1000
19474/19474 [==============================] - 96s 5ms/step - loss: -3026.4192 - accuracy: 0.0101 - val_loss: -5731.0237 - val_accuracy: 0.0103
Epoch 2/1000
19474/19474 [==============================] - 95s 5ms/step - loss: -8420.8756 - accuracy: 0.0101 - val_loss: -11124.1818 - val_accuracy: 0.0103
Epoch 3/1000
19474/19474 [==============================] - 96s 5ms/step - loss: -13836.9917 - accuracy: 0.0101 - val_loss: -16527.2310 - val_accuracy: 0.0103
Epoch 4/1000
19474/19474 [==============================] - 95s 5ms/step - loss: -19231.9505 - accuracy: 0.0101 - val_loss: -21915.0596 - val_accuracy: 0.0103
Epoch 5/1000
13920/19474 [====================>.........] - ETA: 27s - loss: -23830.5972 - accuracy: 0.0101

KeyboardInterrupt: 